Optimisation de l'entrainement pour `focus` 
This is the same function as used in `10_Transfer_learning_what_networks.ipynb`
> ... TODO ... # TODO test without circular padding, with Adam, with no warmstart 

    model = torchvision.models.resnet18(weights=None)

# optimize meta-parameters

In [ ]:
# print_gpu_memory()

# print(path_save)
# %ls -l {path}*

# %rm {path} + '.sqlite3'  # FORCING RECOMPUTE

In [ ]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
model_name = 'resnet101'
dataset = 'bbox' # Select your root between : 'boxes', 'full'
dataset = 'full'

model_filename = f'32_fovea_model_name={model_name}_dataset={dataset}_retrained.pth'
model_filename = None # do not use resumed net
scan_dicts= {'delta2' : [0., 0.001]  }
label_dicts= { 'lr' : 'lr',  'delta2' : '1 - beta2', 'mutnemom' : 'mutnemom'}

In [ ]:
import retinoto_py as fovea
batch_size = 64
opts_dict = dict(do_fovea=True, model_name=model_name, 
                 num_epochs=5,
                 n_train_stop=256*batch_size, n_val_stop=64*batch_size, )
args = fovea.Params(**opts_dict)
args

In [ ]:
TRAIN_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'train'
train_dataset, class_to_idx, idx_to_class = fovea.get_dataset(args, TRAIN_DATA_DIR, n_stop=args.n_train_stop) #, in_memory=False)
train_loader = fovea.get_loader(args, train_dataset)
VAL_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'val'
val_dataset, class_to_idx, idx_to_class = fovea.get_dataset(args, VAL_DATA_DIR, n_stop=args.n_val_stop)
val_loader = fovea.get_loader(args, val_dataset)

In [ ]:
from torchvision import datasets
ds = datasets.ImageFolder(root=VAL_DATA_DIR, transform=None)

In [ ]:
len(ds.imgs)

In [ ]:
%ls -l {model_filename}

In [ ]:
# subplotpars_scan = SubplotParams(left=0.125, right=.95, bottom=0.25, top=.975)
from dataclasses import asdict, field
# max_threshold = .999
for key in scan_dicts:
   
    print(50*'=')
    print('Scanning along', key, "=", label_dicts[key])
    print(50*'=')
    
    measure_columns = [key, 'accuracy']
    i_loc = 0
    for i_value, value in enumerate(scan_dicts[key]):
        filename = args.data_cache / f'34_optuna_{key}_{i_value}.json'
        if filename.is_file():
            df_scan = fovea.pd.read_json(filename)
        else:
            df_scan = fovea.pd.DataFrame([], columns=measure_columns)
            print('i_value', i_value + 1, ' /', len(scan_dicts[key]), key, '=', value)

            opt = fovea.Params(*opts_dict)

            new_dict = asdict(opt)
            new_dict[key] = value
            new_opt = fovea.Params(**new_dict)
            
            def objective(trial):
                    
                # # new_opt.batch_size = trial.suggest_int('batch_size', 16, 512, log=True, step=1)
                # # new_opt.rs_min = trial.suggest_float('rs_min', -1, 1.)
                # # new_opt.rs_max = trial.suggest_float('rs_max', -7, -4)
                # new_opt.mutnemom = trial.suggest_float('mutnemom', 0.01, 0.2, log=True)
                scale = 10
                if new_opt.delta2>0: new_opt.delta2 = trial.suggest_float('delta2', 0.001, 0.9, log=True)
                new_opt.weight_decay = trial.suggest_float('weight_decay', opt.weight_decay / scale, opt.weight_decay * scale, log=True)
                new_opt.label_smoothing = trial.suggest_float('label_smoothing', opt.label_smoothing / scale, opt.label_smoothing * scale, log=True)

                scale = 100 
                new_opt.lr_class = trial.suggest_float('lr', opt.lr / scale, opt.lr * scale, log=True)
                # new_opt.im_mean = trial.suggest_float('im_mean', opt.im_mean / scale, opt.im_mean * scale, log=True)
                # new_opt.im_std = trial.suggest_float('im_std', opt.im_std / scale, opt.im_std * scale, log=True)

                # get the architecture of the network
                model = fovea.load_model(args, model_path=model_filename)

                # train and get accuracy on the validation set
                model_retrain, df_train = fovea.train_model(args, model=model, train_loader=train_loader, val_loader=val_loader)

                accuracy = df_train['acc_val'].mean()
                
                return accuracy
                
            # 3. Create a study object and optimize the objective function.
            sampler = optuna.samplers.TPESampler(multivariate=True, warn_independent_sampling=False)
            opt_tuna= dict(storage=f"sqlite:///cached_data/34_optuna.sqlite3", 
                           sampler=sampler, direction='maximize', load_if_exists=True, study_name=f"{key} = {value}")
            study = optuna.create_study(**opt_tuna)
            print(f'Starting optimization for {key} = {value} with params: {new_opt} on {max(150-len(study.trials), 0)} trials / {len(study.trials)} ')
            study.optimize(objective, n_trials=max((150-len(study.trials), 0)), n_jobs=1, show_progress_bar=True)

            print(50*'-.')
            print("Best params: ", study.best_params)
            print("Best value: ", study.best_value)
            print("Best Trial: ", study.best_trial)
            # print("Trials: ", study.trials)
            print(50*'-.')
            df_scan.loc[i_loc] = {key:value, 'accuracy':study.best_value}
            # print_gpu_memory()

            df_scan.to_json(filename, orient='index', indent=2)

        print(df_scan)
        print(50*'=')

In [ ]:
import optuna.visualization.matplotlib as vis

In [ ]:
vis.plot_param_importances(study)

In [ ]:
# df = study.trials_dataframe()   
params = sorted({k for t in study.trials for k in t.params})
params

In [ ]:

fig, axes = fovea.plt.subplots(len(params), 1, figsize=(15, 8*len(params)), sharey=True)

for ax, pname in zip(axes, params):
    xs = [t.params[pname] for t in study.trials if pname in t.params]
    ys = [t.value for t in study.trials if pname in t.params]
    ax.scatter(xs, ys, s=20, alpha=0.6)
    ax.set_xlabel(pname)
    ax.set_ylabel("Objective")

fovea.plt.tight_layout()


In [ ]:
# vis.plot_contour(study, params=["weight_quanta", "resample_rate"])
# vis.plot_contour(study, params=["num_particles", "chunk_size"])